In [54]:
from pyspark.sql import SparkSession


In [55]:
spark = SparkSession.builder.appName("Political_data").config("spark.sql.caseSensitive", "True").getOrCreate()


In [ ]:
spark

In [57]:
db=spark.read.json("donation_np.json")


In [ ]:
# db.withColumnRenamed("Contribution Mode","mode_of_payment").show() #single line change
db=db.withColumnRenamed("Contribution Mode","mode_of_payment")
db=db.withColumnRenamed("Financial Year","fin_year")
db=db.withColumnRenamed("PAN Given","pan_given")
db.show()


•	Change below column names:

Contribution Mode	mode_of_payment
Financial Year	fin_year
PAN Given	pan_given


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import hashlib
def encrypt_value(mobno):
    sha_value = hashlib.sha256(mobno.encode()).hexdigest()
    return sha_value

spark_udf = udf(encrypt_value, StringType())
#data = db.withColumn('encrypted_value',spark_udf('Address'))
db.withColumn('Address',md5(col("Address"))).show()


•	Encrypt address column

16-B, Ferozeshah Road New Delhi-1	aebd8d41127096039df138069fab7630


•	Change below column names:

Contribution Mode	mode_of_payment
Financial Year	fin_year
PAN Given	pan_given


In [ ]:
db = db.withColumn("payment_seperated", when(lower(db["mode_of_payment"]).contains('cash'), 'Cash').when(lower(db["mode_of_payment"]).contains('bank'), 'Bank').when(lower(db["mode_of_payment"]).contains('cheque') | lower(db["mode_of_payment"]).contains('ch.') | lower(db["mode_of_payment"]).contains('ch no'), 'Cheque').otherwise('Others'))
db.show()

•	Categorize mode_of_payment into below 4 categories
o	Cash
o	Cheque
o	Bank
o	Others


In [ ]:
modeArray=[]
INC_SUM=0
BJP_SUM=0
NCP_SUM=0
CPI_SUM=0
CPIM_SUM=0

INC_COUNT=0
BJP_COUNT=0
NCP_COUNT=0
CPI_COUNT=0
CPIM_COUNT=0

INC_MAX=0
BJP_MAX=0
NCP_MAX=0
CPI_MAX=0
CPIM_MAX=0


for mode in db.select("Party","Amount").collect():
   if mode[0]=="INC":
      INC_SUM=INC_SUM+int(mode[1])
      INC_COUNT=INC_COUNT+1
      if int(mode[1])>INC_MAX:
         INC_MAX=int(mode[1])

   if mode[0]=="BJP":
      BJP_SUM=BJP_SUM+int(mode[1])
      BJP_COUNT=BJP_COUNT+1
      if int(mode[1])>BJP_MAX:
         BJP_MAX=int(mode[1])

   if mode[0]=="NCP":
      NCP_SUM=NCP_SUM+int(mode[1])
      NCP_COUNT=NCP_COUNT+1
      if int(mode[1])>NCP_MAX:
         NCP_MAX=int(mode[1])


   if mode[0]=="CPI":
      CPI_SUM=CPI_SUM+int(mode[1])
      CPI_COUNT=CPI_COUNT+1
      if int(mode[1])>CPI_MAX:
         CPI_MAX=int(mode[1])


   if mode[0]=="CPI(M)":
      CPIM_SUM=CPIM_SUM+int(mode[1])
      CPIM_COUNT=CPIM_COUNT+1
      if int(mode[1])>CPIM_MAX:
         CPIM_MAX=int(mode[1])
   
db=db.withColumn("INC_SUM_LTD", \
    when((db.Party == "INC"), lit(INC_SUM)) \
      .otherwise(lit(0)) \
 )


db=db.withColumn("BJP_SUM_LTD", \
    when((db.Party == "BJP"), lit(BJP_SUM)) \
      .otherwise(lit(0)) \
 )

db=db.withColumn("NCP_SUM_LTD", \
    when((db.Party == "NCP"), lit(NCP_SUM)) \
      .otherwise(lit(0)) \
 )

db=db.withColumn("CPI_SUM_LTD", \
    when((db.Party == "CPI"), lit(CPI_SUM)) \
      .otherwise(lit(0)) \
 )

db=db.withColumn("'CPI(M)_SUM_LTD", \
    when((db.Party == "CPI(M)"), lit(CPIM_SUM)) \
      .otherwise(lit(0)) \
 )


INC_AVG=INC_SUM//INC_COUNT
BJP_AVG=BJP_SUM//BJP_COUNT
NCP_AVG=NCP_SUM//NCP_COUNT
CPI_AVG=CPI_SUM//CPI_COUNT
CPIM_AVG=CPIM_SUM//CPIM_COUNT

db.show()

•	Calculate aggregates per party
(below columns should b)

'INC_SUM_LTD',
 'BJP_SUM_LTD',
 'NCP_SUM_LTD',
 'CPI_SUM_LTD',
 'CPI(M)_SUM_LTD',


In [103]:
from pyspark.sql.types import IntegerType

db = db.withColumn("Amount",col("Amount").cast(IntegerType()))

new = db.groupBy('fin_year','Party').sum("Amount")

for i,j,k in new.select('fin_year','Party',"sum(Amount)").collect():

  db = db.withColumn("{}_{}_SUM".format(i,j),when( (db.Party == "{}".format(j)) & (db.fin_year == "{}".format(i)) ,lit(k) ).otherwise(lit(0)))

•	Calculate sum of donations per financial year per party and the	 column should be named as 2011-12_BJP_SUM

In [ ]:
db=db.withColumn("INC_COUNT_LTD",when((db.Party == "INC"), lit(INC_COUNT)) .otherwise(lit(0)))
db=db.withColumn("BJP_COUNT_LTD",when((db.Party == "BJP"), lit(BJP_COUNT)) .otherwise(lit(0)))
db=db.withColumn("NCP_COUNT_LTD",when((db.Party == "NCP"), lit(NCP_COUNT)) .otherwise(lit(0)))
db=db.withColumn("CPI_COUNT_LTD",when((db.Party == "CPI"), lit(CPI_COUNT)) .otherwise(lit(0)))
db=db.withColumn("CPIM_COUNT_LTD",when((db.Party == "CPIM"), lit(CPIM_COUNT)) .otherwise(lit(0)))

db=db.withColumn("INC_AVG_LTD",when((db.Party == "INC"), lit(INC_AVG)) .otherwise(lit(0)))
db=db.withColumn("BJP_AVG_LTD",when((db.Party == "BJP"), lit(BJP_AVG)) .otherwise(lit(0)))
db=db.withColumn("NCP_AVG_LTD",when((db.Party == "NCP"), lit(NCP_AVG)) .otherwise(lit(0)))
db=db.withColumn("CPI_AVG_LTD",when((db.Party == "CPI"), lit(CPI_AVG)) .otherwise(lit(0)))
db=db.withColumn("CPIM_AVG_LTD",when((db.Party == "CPIM"), lit(CPIM_AVG)) .otherwise(lit(0)))
db.show()

In [101]:
BJP_TOP_DONOR=BJP_MAX
CPI_TOP_DONOR=CPI_MAX
INC_TOP_DONOR=INC_MAX
NCP_TOP_DONOR=NCP_MAX
CPIM_TOP_DONOR=CPIM_MAX


•	Calculate top donor per party 

o	BJP_TOP_DONOR
o	CPI_TOP_DONOR
o	INC_TOP_DONOR
o	NCP_TOP_DONOR
o	CPI(M)_TOP_DONOR


In [ ]:
Bank_count=0
Cheque_count=0
Cash_count=0
Others_count=0

for payment_seperated in db.select("payment_seperated").collect():
    if payment_seperated=="Bank":
        Bank_count=Bank_count+1
    elif payment_seperated=="Cheque":
        Cheque_count=Cheque_count+1
    elif payment_seperated=="Cash":
        Cash_count=Cash_count+1
    else:
        Others_count=Others_count+1

db=db.withColumn("Bank_count_LTD",when((db.payment_seperated == "Bank"), lit(Bank_count)) .otherwise(lit(0)))
db=db.withColumn("Cheque_count_LTD",when((db.payment_seperated == "Cheque"), lit(Cheque_count)) .otherwise(lit(0)))
db=db.withColumn("Cash_count_LTD",when((db.payment_seperated == "Cash"), lit(Cash_count)) .otherwise(lit(0)))
db=db.withColumn("Others_count_LTD",when((db.payment_seperated == "Others"), lit(Others_count)) .otherwise(lit(0)))

db.show()

•	Calculate number of donation till date per mode_of_payment
o	Bank_count_LTD
o	Cheque_count_LTD
o	Cash_count_LTD
o	Others_count_LTD
